In [24]:
#reference : https://www.zenrows.com/blog/dynamic-web-pages-scraping-python#selecting-elements
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import io


In [2]:

driver = webdriver.Chrome(ChromeDriverManager().install())
url = 'https://loveracing.nz/RaceInfo/Jockey-Premierships.aspx'
driver.get(url)


In [4]:


print(driver.page_source)

<html class=" js flexbox canvas canvastext webgl no-touch geolocation postmessage no-websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths -webkit- yui3-js-enabled" lang="en"><div id="yui3-css-stamp" style="position: absolute !important; visibility: hidden !important" class=""></div><!--<![endif]--><head>
<title>Jockey Premierships | RaceInfo </title>
<meta id="keywords" name="keywords" content="racing, thoroughbreds, horses, horse racing">
<meta id="description" name="description" content="New Zealand Thoroughbred Racing’s premium digital channel showcasing horse racing.  We deliver the latest news, video, profiles, field &amp; form, statistics and raceday inf

# Can `read_html` from pandas, deal with the table for us?

In [ ]:
#checking if I can get the table in a easier way
import requests
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

pd.read_html(r.text)

## The answer is no, at least not as easy as it should, so what now?

In [13]:

from selenium.webdriver.common.by import By
elements = driver.find_elements(By.CLASS_NAME, 'tables') 
trainers_names = [trainers_elements.text for trainers_elements in elements]
#print(trainers_names)
trainers_names


["JOCKEYS\nJUMPING JOCKEYS\nPremierships are based on performances during each racing season (August 1st - July 31st). The jockey and trainer with the most wins at the end of the season are the Premiership winners. These statistics are updated daily.\n    Wins 2nds 3rds Stakes Starts G & L SR\n1 W Kennedy 80 46 38 $3,895,367 423 14 5.29\n2 M McNab 72 65 54 $2,488,137 383 2 5.32\n3 J Doyle 63 69 64 $3,079,045 550 6 8.73\n4 O Bosson 56 30 21 $2,798,640 213 8 3.8\n5 C Grylls 50 44 55 $2,634,532 409 4 8.18\n6 A Comignaghi 46 55 39 $1,473,055 327 2 7.11\n7 L Allpress 44 39 40 $1,514,332 360 2 8.18\n8 S Spratt 37 40 36 $1,919,945 364 4 9.84\n9 K Asano 37 34 30 $1,223,717 354 2 9.57\n10 J Fawcett 37 31 20 $1,104,920 306 1 8.27\n11 S Weatherley 35 35 27 $1,577,180 281 2 8.03\n12 L Sutherland (a) 33 25 29 $1,235,170 339 3 10.27\n13 J Parkes 30 15 11 $798,665 185 2 6.17\n14 K Williams 29 27 46 $841,025 274 1 9.45\n15 K Hercock 28 31 36 $1,038,385 323 0 11.54\n16 N Parmar (a) 28 25 21 $1,033,325 


 Let's: 
 1. remove the header and footer text that is not part of the table, the table starts with "1 W Kennedy" and ends with "2 0 2".
 2. replace new lines and multiple spaces to get it clean
 3. simulate a file-like object
 4. read_csv

In [34]:
table_start = trainers_names[0].find("1 W Kennedy")
table_end = trainers_names[0].rfind("2 0 2") + len("2 0 2")
table_trainers_names = trainers_names[0][table_start:table_end]

# Replace newlines and multiple spaces with a single newline to clean up the trainers_names
teste = table_trainers_names.split('\n')

In [52]:
trainers_names

["JOCKEYS\nJUMPING JOCKEYS\nPremierships are based on performances during each racing season (August 1st - July 31st). The jockey and trainer with the most wins at the end of the season are the Premiership winners. These statistics are updated daily.\n    Wins 2nds 3rds Stakes Starts G & L SR\n1 W Kennedy 80 46 38 $3,895,367 423 14 5.29\n2 M McNab 72 65 54 $2,488,137 383 2 5.32\n3 J Doyle 63 69 64 $3,079,045 550 6 8.73\n4 O Bosson 56 30 21 $2,798,640 213 8 3.8\n5 C Grylls 50 44 55 $2,634,532 409 4 8.18\n6 A Comignaghi 46 55 39 $1,473,055 327 2 7.11\n7 L Allpress 44 39 40 $1,514,332 360 2 8.18\n8 S Spratt 37 40 36 $1,919,945 364 4 9.84\n9 K Asano 37 34 30 $1,223,717 354 2 9.57\n10 J Fawcett 37 31 20 $1,104,920 306 1 8.27\n11 S Weatherley 35 35 27 $1,577,180 281 2 8.03\n12 L Sutherland (a) 33 25 29 $1,235,170 339 3 10.27\n13 J Parkes 30 15 11 $798,665 185 2 6.17\n14 K Williams 29 27 46 $841,025 274 1 9.45\n15 K Hercock 28 31 36 $1,038,385 323 0 11.54\n16 N Parmar (a) 28 25 21 $1,033,325 

In [47]:
df =  pd.DataFrame(columns=['Rank', 'Jockey', 'Wins', '2nds', '3rds', 'Stakes', 'Starts', 'G&L', 'SR'])

In [50]:
teste[1].split()

['2', 'M', 'McNab', '72', '65', '54', '$2,488,137', '383', '2', '5.32']

In [49]:
#teste[0].split()
for row in range(0,1):
    #lets append the data row by row
    row_to_append = teste[row].split()
    #get a look at one of the lines 
    if row == 0:
        print(row_to_append)
    length = len(df)
    df.loc[length] = row_to_append

['1', 'W', 'Kennedy', '80', '46', '38', '$3,895,367', '423', '14', '5.29']


ValueError: cannot set a row with mismatched columns

In [32]:


table_start = trainers_names[0].find("1 W Kennedy")
table_end = trainers_names[0].rfind("2 0 2") + len("2 0 2")
table_trainers_names = trainers_names[0][table_start:table_end]

# Replace newlines and multiple spaces with a single newline to clean up the trainers_names
table_trainers_names = '\n'.join(' '.join(line.split()) for line in table_trainers_names.split('\n'))

# Simulate a file-like object
table_io = io.StringIO(table_trainers_names)

# Assuming the trainers_names is space-separated and the first column is the rank number
# You might need to adjust the `sep` parameter if your trainers_names uses a different delimiter
df = pd.read_csv(table_io, sep=r'\s{2,}', engine='python', names=[
    'Rank', 'Jockey', 'Wins', '2nds', '3rds', 'Stakes', 'Starts', 'G&L', 'SR'
])

# Now `df` is your dataFrame with the table trainers_names
print(df)


                                            Rank  Jockey  Wins  2nds  3rds  \
0    1 W Kennedy 80 46 38 $3,895,367 423 14 5.29     NaN   NaN   NaN   NaN   
1       2 M McNab 72 65 54 $2,488,137 383 2 5.32     NaN   NaN   NaN   NaN   
2       3 J Doyle 63 69 64 $3,079,045 550 6 8.73     NaN   NaN   NaN   NaN   
3       4 O Bosson 56 30 21 $2,798,640 213 8 3.8     NaN   NaN   NaN   NaN   
4      5 C Grylls 50 44 55 $2,634,532 409 4 8.18     NaN   NaN   NaN   NaN   
..                                           ...     ...   ...   ...   ...   
99          100 J R Collett 1 0 0 $148,400 8 1 8     NaN   NaN   NaN   NaN   
100         101 A Scott (am) 1 0 0 $11,340 4 0 4     NaN   NaN   NaN   NaN   
101       102 H Hassman (am) 1 0 0 $11,225 4 0 4     NaN   NaN   NaN   NaN   
102         103 C Wiles (am) 1 0 0 $10,120 2 0 2     NaN   NaN   NaN   NaN   
103         104 L Kalychurun 1 0 0 $23,600 2 0 2     NaN   NaN   NaN   NaN   

     Stakes  Starts  G&L  SR  
0       NaN     NaN  NaN NaN  
1

In [28]:
trainers_names[0].find("1 W Kennedy")

289